In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.model_selection import train_test_split

stores_total_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_train, stores_val = train_test_split(stores_total_train, test_size=0.2, random_state=0)

In [2]:
from utils import create_geographical_columns, split_plaace_cat
revenue_df = split_plaace_cat(create_geographical_columns(stores_train))
revenue_df_total = split_plaace_cat(create_geographical_columns(stores_total_train))
result_df_val = split_plaace_cat(create_geographical_columns(stores_val))
result_df_test = split_plaace_cat(create_geographical_columns(stores_test))
result_df_test_total = split_plaace_cat(create_geographical_columns(stores_test))

In [3]:
from constants import *
result_df = revenue_df.copy()
revenue_df_dict : dict[str, pd.DataFrame]
revenue_df_dict_total : dict[str, pd.DataFrame]
revenue_df_dict = {}
revenue_df_dict_total = {}
for level in levels:
    grouped_df = revenue_df.groupby([level, 'plaace_cat_1']).mean()['revenue'].to_frame()
    grouped_df_total = revenue_df_total.groupby([level, 'plaace_cat_1']).mean()['revenue'].to_frame()
    grouped_df.rename(columns={'revenue':f'{level}.mean_revenue'}, inplace=True)
    grouped_df_total.rename(columns={'revenue':f'{level}.mean_revenue'}, inplace=True)
    revenue_df_dict[level] = grouped_df
    revenue_df_dict_total[level] = grouped_df_total
    result_df = pd.merge(result_df, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_val = pd.merge(result_df_val, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_test = pd.merge(result_df_test, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_test_total = pd.merge(result_df_test_total, grouped_df_total, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')

In [4]:
result_df =  result_df[[f'{level}.mean_revenue' for level in levels]+['store_id', 'revenue']]
result_df_val =  result_df_val[[f'{level}.mean_revenue' for level in levels]+['store_id', 'revenue']]
result_df_test =  result_df_test[[f'{level}.mean_revenue' for level in levels]+['store_id']]
result_df_test_total =  result_df_test_total[[f'{level}.mean_revenue' for level in levels]+['store_id']]

In [5]:
result_df_val.shape

(2572, 6)

In [6]:
result_df_val.isna().sum()

grunnkrets_id.mean_revenue    818
delomrade.mean_revenue        227
kommune.mean_revenue           54
fylke.mean_revenue              0
store_id                        0
revenue                         0
dtype: int64

In [7]:
result_df_test.isna().sum()

grunnkrets_id.mean_revenue    2864
delomrade.mean_revenue         833
kommune.mean_revenue           202
fylke.mean_revenue               0
store_id                         0
dtype: int64

In [8]:
result_df_test_total.isna().sum()

grunnkrets_id.mean_revenue    2541
delomrade.mean_revenue         690
kommune.mean_revenue           159
fylke.mean_revenue               0
store_id                         0
dtype: int64

In [9]:
result_df.corr()

,grunnkrets_id.mean_revenue,delomrade.mean_revenue,kommune.mean_revenue,fylke.mean_revenue,revenue
grunnkrets_id.mean_revenue,1.000000,0.727092,0.489814,0.383120,0.727744
delomrade.mean_revenue,0.727092,1.000000,0.673662,0.526921,0.529137
kommune.mean_revenue,0.489814,0.673662,1.000000,0.782175,0.356459
fylke.mean_revenue,0.383120,0.526921,0.782175,1.000000,0.278813
revenue,0.727744,0.529137,0.356459,0.278813,1.000000


In [10]:
result_df_val.corr()

,grunnkrets_id.mean_revenue,delomrade.mean_revenue,kommune.mean_revenue,fylke.mean_revenue,revenue
grunnkrets_id.mean_revenue,1.000000,0.704734,0.490257,0.377568,0.126271
delomrade.mean_revenue,0.704734,1.000000,0.598586,0.476958,0.111950
kommune.mean_revenue,0.490257,0.598586,1.000000,0.811830,0.190145
fylke.mean_revenue,0.377568,0.476958,0.811830,1.000000,0.257962
revenue,0.126271,0.111950,0.190145,0.257962,1.000000


In [11]:
revenue_df_dict

{'grunnkrets_id':                             grunnkrets_id.mean_revenue
 grunnkrets_id plaace_cat_1                            
 1010105       1                               5.160000
               2                               7.286429
               3                               4.202333
 1010107       3                               1.628000
 1010204       1                               4.304000
 ...                                                ...
 20300303      2                               5.268667
               3                               3.322000
 20300309      3                               2.948000
 20300310      1                               8.584000
               2                              31.833000
 
 [5085 rows x 1 columns],
 'delomrade':                         delomrade.mean_revenue
 delomrade plaace_cat_1                        
 10101     1                           5.160000
           2                           7.286429
           3          

In [12]:
for level, df in revenue_df_dict.items():
    df.to_csv(f'temp_data/{level}_mean_revenue.csv')

In [13]:
for level, df in revenue_df_dict_total.items():
    df.to_csv(f'temp_data/{level}_mean_revenue_total.csv')

In [33]:
from avg_revenue import add_avg_revenue, create_avg_revenue_csvs

create_avg_revenue_csvs()
result = add_avg_revenue(stores_val, total=False)

In [34]:
result.isna().sum()

store_id                               0
year                                   0
store_name                             0
plaace_hierarchy_id                    0
sales_channel_name                     0
grunnkrets_id                          0
address                              349
lat                                    0
lon                                    0
chain_name                          1815
mall_name                           2081
revenue                                0
fylke                                  0
kommune                                0
delomrade                              0
plaace_cat_1                           0
plaace_cat_2                           0
plaace_cat_3                           0
plaace_cat_4                           0
grunnkrets_id.mean_revenue             0
grunnkrets_id.mean_revenue_count       0
delomrade.mean_revenue                 0
delomrade.mean_revenue_count           0
kommune.mean_revenue                   0
kommune.mean_rev

In [35]:
result[result['grunnkrets_id.mean_revenue_count'].isna()]

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,...,plaace_cat_3,plaace_cat_4,grunnkrets_id.mean_revenue,grunnkrets_id.mean_revenue_count,delomrade.mean_revenue,delomrade.mean_revenue_count,kommune.mean_revenue,kommune.mean_revenue_count,fylke.mean_revenue,fylke.mean_revenue_count


In [36]:
result[[f'{level}.mean_revenue' for level in levels]+['revenue']].corr()

,grunnkrets_id.mean_revenue,delomrade.mean_revenue,kommune.mean_revenue,fylke.mean_revenue,revenue
grunnkrets_id.mean_revenue,1.000000,0.668862,0.468267,0.396810,0.715079
delomrade.mean_revenue,0.668862,1.000000,0.668876,0.571474,0.478664
kommune.mean_revenue,0.468267,0.668876,1.000000,0.849542,0.318923
fylke.mean_revenue,0.396810,0.571474,0.849542,1.000000,0.271024
revenue,0.715079,0.478664,0.318923,0.271024,1.000000
